In [1]:
import pandas as pd
import numpy as np
import time
import re
from tqdm import tqdm_notebook
from torrequest import TorRequest
from bs4 import BeautifulSoup
from retrying import retry
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Danh sách các cầu thủ đã được thu thập trước đó
sampled_players = pd.read_csv('C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Obtaining_players_and_teams/players_transfermarket.csv')

In [3]:
def get_soup(URL):
    
    chrome_options = Options()
    chrome_options.add_argument('--window-size=1,1')
    
    #options = webdriver.ChromeOptions()
    #options.add_argument('--disable-popup-blocking')
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)
    wait = WebDriverWait(driver, 10)
    driver.get(URL)
    if len(driver.find_elements_by_css_selector("[name=ROBOTS]")) > 0:
        driver.get(URL)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return soup

In [4]:
print(sampled_players.shape)

(10754, 4)


In [7]:
def extract_player_info_block(players_df):
    
    #links = players_df.link_ws
    #teams = players_df.team
    
    player_info = {'name': [],
                  'position': [],
                  'height': [],
                  'age': [],
                  'team': []} 

    for index in range(players_df.shape[0]):
        URL = 'https://www.transfermarkt.co.uk' + players_df.iloc[index]['link']
        print(URL)
        soup = get_soup(URL)
        text = ""
        div = soup.find('div', class_ = 'info-table info-table--right-space')
        
        if div is None:
            #soup = get_soup(URL)
            div = soup.find('div', class_ = 'info-table info-table--right-space min-height-audio')
            
        key_list = div.find_all('span', class_ = 'info-table__content info-table__content--regular')
        value_list = div.find_all('span', class_ = 'info-table__content info-table__content--bold')
        
        
        # Get the values
        h = 0
        a = 0
        p = 0
        for i in range(len(key_list)):
            if key_list[i].text == "Height:":
                # Preprocess the height of the player
                height_str = value_list[i].text
                height_str = height_str.replace(',', '.')  # Replace comma with period for decimal separator
                height_str = height_str.replace('\xa0', '')  # Remove non-breaking space character
                height_str = height_str.replace('m', '')  # Remove the unit 'm'
                height_str = height_str.replace('،', '.')
                height = float(height_str)
                height = int(height * 100)  # Convert to centimeters
                player_info['height'].append(height)
                h = 1
            elif key_list[i].text == "Age:":
                age = int(value_list[i].text)
                player_info['age'].append(age)
                a = 1
            elif key_list[i].text == "Position:":
                # Preprocess the position of the player
                pos = value_list[i].text
                pos = pos.replace(' ', '')
                pos = pos[1:]
                if pos.count("-") >= 2:
                    pos = pos.replace("-", " ", 1)
                player_info['position'].append(pos)
                p = 1

        player_info['name'].append(players_df.iloc[index]['player'])
        player_info['team'].append(players_df.iloc[index]['team'])
        
        if h == 0:
            player_info['height'].append(None)
        if a == 0:
            player_info['age'].append(None)
        if p == 0:
            player_info['position'].append(None)

            
        print(index, "Complete", players_df.iloc[index]['player'])
        
        '''    
            try:
                text = text + str(each)
            except:
                None
        
        name = re.findall(r'((?<=Name:</dt>\n<dd>).+(?=</dd>\n</dl><dl class))', text)
        height = re.findall(r'((?<=Height:</dt>\n<dd>).+(?=cm</dd>))', text)
        position = re.findall(r'((?<=Positions:</dt>\n<dd>\n<ul>\n<li>).*(?= |</li>))', text)
        weight = re.findall(r'((?<=Weight:</dt>\n<dd>).+(?=kg))', text)
        
        player_info['name'].append(name[0]) if len(name) >0 else player_info['name'].append(np.nan)
        player_info['position'].append(position[0].strip()) if len(position) >0 else player_info['position'].append(np.nan)
        player_info['height'].append(height[0]) if len(height) >0 else player_info['height'].append(np.nan)
        player_info['weight'].append(weight[0]) if len(weight) >0 else player_info['weight'].append(np.nan)
        player_info['team'].append(each_team)
        '''


    player_info_df = pd.DataFrame.from_dict(player_info, orient='index')
    player_info_df = player_info_df.transpose()
    
    return player_info_df

In [8]:
# Vì danh sách cầu thủ nhiều, nếu cào hết 1 lần rất tốn tài nguyên 
# -> chia data thành các sample nhỏ để dễ dàng crawl thông tin của từng cầu thủ hơn
# Sample_data là một list, mỗi phần tử là 1 dataframe tách ra từ sampled_players
sample_data = []
sample_data.append(sampled_players.iloc[:10])
i = 10
while i < sampled_players.shape[0]:
    if i >= sampled_players.shape[0] - 200:
        sample_data.append(sampled_players.iloc[i:])
    else:
        sample_data.append(sampled_players.iloc[i:i+200])
    i += 200 # mỗi sample chứa 200 dòng
print(len(sample_data))

54


In [19]:
print(sample_data[48])

      Unnamed: 0                                   link          player  \
9700          34   /sung-won-jang/profil/spieler/606411   Sung-won Jang   
9701          35   /yong-hui-park/profil/spieler/992530   Yong-hui Park   
9702          36    /jong-hun-lee/profil/spieler/638562    Jong-hun Lee   
9703          37   /dong-hyun-kim/profil/spieler/970946   Dong-hyun Kim   
9704          38      /edgar-bruno/profil/spieler/45673     Edgar Bruno   
...          ...                                    ...             ...   
9895          24     /won-sang-um/profil/spieler/630269     Won-sang Um   
9896          25   /chung-yong-lee/profil/spieler/81801  Chung-yong Lee   
9897          26  /jae-hwan-hwang/profil/spieler/639244  Jae-hwan Hwang   
9898          27  /see-young-jang/profil/spieler/858644  See-young Jang   
9899          28     /martin-adam/profil/spieler/220635     Martin Ádám   

               team  
9700       Daegu FC  
9701       Daegu FC  
9702       Daegu FC  
9703       

In [22]:
# Muốn cào sample nào thì thay i bàng sample đó
i = 39
sample = extract_player_info_block(sample_data[i])

https://www.transfermarkt.co.uk/joaquin-varela/profil/spieler/421194
0 Complete Joaquín Varela
https://www.transfermarkt.co.uk/ezequiel-parnisiari/profil/spieler/210742
1 Complete Ezequiel Parnisiari
https://www.transfermarkt.co.uk/jonathan-bay/profil/spieler/360425
2 Complete Jonathan Bay
https://www.transfermarkt.co.uk/sebastian-corda/profil/spieler/843122
3 Complete Sebastián Corda
https://www.transfermarkt.co.uk/giuliano-cerato/profil/spieler/971423
4 Complete Giuliano Cerato
https://www.transfermarkt.co.uk/oscar-garrido/profil/spieler/840954
5 Complete Oscar Garrido
https://www.transfermarkt.co.uk/juan-franco/profil/spieler/331245
6 Complete Juan Franco
https://www.transfermarkt.co.uk/nicolas-linares/profil/spieler/534616
7 Complete Nicolás Linares
https://www.transfermarkt.co.uk/nicolas-watson/profil/spieler/697045
8 Complete Nicolás Watson
https://www.transfermarkt.co.uk/roberto-bochi/profil/spieler/267387
9 Complete Roberto Bochi
https://www.transfermarkt.co.uk/gaston-lodico/pr

85 Complete Guido Villar
https://www.transfermarkt.co.uk/rodrigo-herrera/profil/spieler/842141
86 Complete Rodrigo Herrera
https://www.transfermarkt.co.uk/juan-ignacio-diaz/profil/spieler/535028
87 Complete Juan Ignacio Díaz
https://www.transfermarkt.co.uk/francisco-alvarez/profil/spieler/565766
88 Complete Francisco Álvarez
https://www.transfermarkt.co.uk/agustin-dattola/profil/spieler/865490
89 Complete Agustín Dattola
https://www.transfermarkt.co.uk/nicolas-capraro/profil/spieler/992818
90 Complete Nicolás Capraro
https://www.transfermarkt.co.uk/brian-calderara/profil/spieler/830111
91 Complete Brian Calderara
https://www.transfermarkt.co.uk/rodrigo-insua/profil/spieler/721235
92 Complete Rodrigo Insúa
https://www.transfermarkt.co.uk/fernando-prado/profil/spieler/836421
93 Complete Fernando Prado
https://www.transfermarkt.co.uk/nicolas-tolosa/profil/spieler/1103929
94 Complete Nicolás Tolosa
https://www.transfermarkt.co.uk/mauro-peinipil/profil/spieler/989571
95 Complete Mauro Peini

170 Complete Érik Godoy
https://www.transfermarkt.co.uk/nicolas-meriano/profil/spieler/941511
171 Complete Nicolás Meriano
https://www.transfermarkt.co.uk/francisco-oliver/profil/spieler/486025
172 Complete Francisco Oliver
https://www.transfermarkt.co.uk/diego-novaretti/profil/spieler/87493
173 Complete Diego Novaretti
https://www.transfermarkt.co.uk/alejandro-rebola/profil/spieler/88080
174 Complete Alejandro Rébola
https://www.transfermarkt.co.uk/lucas-diarte/profil/spieler/283386
175 Complete Lucas Diarte
https://www.transfermarkt.co.uk/alex-ibacache/profil/spieler/571083
176 Complete Alex Ibacache
https://www.transfermarkt.co.uk/juan-barinaga/profil/spieler/701491
177 Complete Juan Barinaga
https://www.transfermarkt.co.uk/gabriel-compagnucci/profil/spieler/538608
178 Complete Gabriel Compagnucci
https://www.transfermarkt.co.uk/santiago-longo/profil/spieler/697065
179 Complete Santiago Longo
https://www.transfermarkt.co.uk/ivan-ramirez/profil/spieler/531093
180 Complete Iván Ramíre

In [23]:
print(sample)

                     name               position height age  \
0          Joaquín Varela   Defender Centre-Back    189  24   
1     Ezequiel Parnisiari   Defender Centre-Back    184  33   
2            Jonathan Bay     Defender Left-Back    179  30   
3         Sebastián Corda     Defender Left-Back    170  27   
4         Giuliano Cerato    Defender Right-Back    170  24   
..                    ...                    ...    ...  ..   
195    Joaquín Susvielles  Attack Centre-Forward    186  32   
196   Alexis Martín Arias             Goalkeeper    185  30   
197  Federico Lanzillotta             Goalkeeper    185  30   
198   Miguel Ángel Acosta             Goalkeeper    189  21   
199    Kevin Mac Allister   Defender Centre-Back    172  25   

                       team  
0      Instituto AC Córdoba  
1      Instituto AC Córdoba  
2      Instituto AC Córdoba  
3      Instituto AC Córdoba  
4      Instituto AC Córdoba  
..                      ...  
195  Club Atlético Belgrano  
196

In [24]:
# Lưu thành file sample_data_i.csv
file_name = "C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Basic_info_player/sample_data_" + str(i) + ".csv"
sample.to_csv(file_name) 